In [1]:
import pandas as pd
import torch
import numpy as np
from scipy.stats import spearmanr
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [2]:
log = pd.read_pickle("E:\projetos\Grad-Eclip\\results")

log = {
        'img_id': [],
        'interpretability_method': [],
        'label': [],
        'similarity_original': [],
        'similarity_perturb': [],
        'hm_original': [],
        'hm_perturbed': [],
        'alpha': [],
        'clip_model': []
    }

In [2]:
log.head()

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model
0,0,selfattn,9,0.297058,0.304125,"[[tensor(5.6957e-09), tensor(5.6957e-09), tens...","[[tensor(5.6957e-09), tensor(5.6957e-09), tens...",0.031373,ViT-B/16
1,0,gradcam,9,0.297058,0.304125,"[[tensor(0.2165), tensor(0.2165), tensor(0.216...","[[tensor(0.2165), tensor(0.2165), tensor(0.216...",0.031373,ViT-B/16
2,0,maskclip,9,0.297058,0.304125,"[[tensor(0.1461), tensor(0.1461), tensor(0.146...","[[tensor(0.1461), tensor(0.1461), tensor(0.146...",0.031373,ViT-B/16
3,0,eclip,9,0.297058,0.304125,"[[tensor(0.0619), tensor(0.0619), tensor(0.061...","[[tensor(0.0619), tensor(0.0619), tensor(0.061...",0.031373,ViT-B/16
4,0,game,9,0.297058,0.304125,"[[tensor(0.0140), tensor(0.0140), tensor(0.014...","[[tensor(0.0685), tensor(0.0685), tensor(0.068...",0.031373,ViT-B/16


In [10]:
filtered_data = log.loc[log["interpretability_method"] == "eclip", ["hm_original", "hm_perturbed"]]
filtered_data

,hm_original,hm_perturbed
3,"[[tensor(0.0619), tensor(0.0619), tensor(0.061...","[[tensor(0.0619), tensor(0.0619), tensor(0.061..."
10,"[[tensor(0.0222), tensor(0.0222), tensor(0.022...","[[tensor(0.0222), tensor(0.0222), tensor(0.022..."
17,"[[tensor(0.0244), tensor(0.0244), tensor(0.024...","[[tensor(0.0244), tensor(0.0244), tensor(0.024..."
24,"[[tensor(0.0400), tensor(0.0400), tensor(0.040...","[[tensor(0.0400), tensor(0.0400), tensor(0.040..."
31,"[[tensor(0.0715), tensor(0.0715), tensor(0.071...","[[tensor(0.0715), tensor(0.0715), tensor(0.071..."
38,"[[tensor(0.0410), tensor(0.0410), tensor(0.041...","[[tensor(0.0410), tensor(0.0410), tensor(0.041..."
45,"[[tensor(0.0510), tensor(0.0510), tensor(0.051...","[[tensor(0.0510), tensor(0.0510), tensor(0.051..."
52,"[[tensor(0.0147), tensor(0.0147), tensor(0.014...","[[tensor(0.0147), tensor(0.0147), tensor(0.014..."
59,"[[tensor(0.0413), tensor(0.0413), tensor(0.041...","[[tensor(0.0413), tensor(0.0413), tensor(0.041..."
66,"[[tensor(0.0938), tensor(0.0938), tensor(0.093...","[[tensor(0.0938), tensor(0.0938), tensor(0.093..."


In [11]:
for idx, row in filtered_data.iterrows():
    # Cada heatmap está como lista de listas de tensores -> converter para tensor
    orig = torch.tensor([[t.item() for t in inner] for inner in row["hm_original"]])
    pert = torch.tensor([[t.item() for t in inner] for inner in row["hm_perturbed"]])
    
    # Flatten para 1D arrays
    flat_orig = orig.flatten().to(device).numpy()
    flat_perturb = pert.flatten().to(device).numpy()
    
    # Calcular Spearman
    corr, p_val = spearmanr(flat_orig, flat_perturb)
    print(f"Spearman correlation: {corr:.3f}, p-value: {p_val:.3e}")

Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00
Spearman correlation: 1.000, p-value: 0.000e+00


In [12]:
def topk_intersection(map1, map2, k):
    flat1 = map1.flatten()
    flat2 = map2.flatten()
    
    topk_indices_1 = torch.topk(flat1.clone().detach(), k).indices
    topk_indices_2 = torch.topk(flat2.clone().detach(), k).indices

    set1 = set(topk_indices_1.tolist())
    set2 = set(topk_indices_2.tolist())    
    
    intersection = set1.intersection(set2)
    
    return len(intersection), len(intersection) / k

# Valor de k (pode ajustar)
k = 100

for idx, row in filtered_data.iterrows():
    # Converter para tensores
    map1 = torch.tensor([[t.item() for t in inner] for inner in row["hm_original"]])
    map2 = torch.tensor([[t.item() for t in inner] for inner in row["hm_perturbed"]])
    
    # Calcular interseção
    inter_count, inter_ratio = topk_intersection(map1, map2, k)
    print(f"Top-{k} intersection count: {inter_count}")
    print(f"Top-{k} intersection ratio: {inter_ratio:.3f}") 

Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
